In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import tensorflow as tf
# from tensorflow.keras import layers, losses
# from tensorflow.keras.models import Model
# import keras_tuner as kt
# import modred as mr
from sklearn.decomposition import PCA, KernelPCA
from scipy import linalg
from sklearn.preprocessing import normalize
# from scipy import fftpack
from pathlib import Path
from scipy.optimize import minimize, differential_evolution


In [2]:
mpl.use('Agg')

# density_df_400_2013 = pd.read_csv('Data/2013_HASDM_400-475KM.den', delim_whitespace=True,
#                                   header=None)
# density_np_400_2013 = pd.DataFrame.to_numpy(density_df_400_2013)
# del density_df_400_2013
# If you want to add more altitudes (increase the initial dimension of the problem), uncomment the followings:
density_df_500_2013 = pd.read_csv('Data/2013_HASDM_500-575KM.den', delim_whitespace=True,
                                  header=None)
density_np_500_2013 = pd.DataFrame.to_numpy(density_df_500_2013)
del density_df_500_2013

# If you want to add more data (increase the time instants), uncomment the followings:
# density_df_400_2014 = pd.read_csv('Data/2014_HASDM_400-475KM.den', delim_whitespace=True,
#                                   header=None)
# density_np_400_2014 = pd.DataFrame.to_numpy(density_df_400_2014)
# del density_df_400_2014

nt = 19
nphi = 24

t = np.linspace(-np.pi / 2, np.pi / 2, nt)
phi = np.linspace(0, np.deg2rad(345), nphi)

# max_rho1 = np.max(density_np_400_2013[:, 10])
# max_rho2 = np.max(density_np_400_2014[:, 10])
# max_rho = np.max(np.array([max_rho1, max_rho2]))
# density_np_400_2013[:, 10] = density_np_400_2013[:, 10] / max_rho
# density_np_400_2014[:, 10] = density_np_400_2014[:, 10] / max_rho

# max_rho = np.max(density_np_400_2013[:, 10])
max_rho = np.max(density_np_500_2013[:, 10])
# max_rho = np.max(np.array([max_rho2, max_rho]))

# density_np_400_2013[:, 10] = density_np_400_2013[:, 10] / max_rho
density_np_500_2013[:, 10] = density_np_500_2013[:, 10] / max_rho
# density_np_400_2014[:, 10] = density_np_400_2014[:, 10] / max_rho

rho_list = []
rho_list1 = []
rho_list2 = []

for i in range(int(1331520 / (nt * nphi))):  # 1335168
    # rho_400_i_2013 = density_np_400_2013[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    # rho_polar_400_i_2013 = np.reshape(rho_400_i_2013, (nt, nphi, 4))

    # rho_400_i_2014 = density_np_400_2014[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    # rho_polar_400_i_2014 = np.reshape(rho_400_i_2014, (nt, nphi, 4))

    rho_500_i = density_np_500_2013[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    rho_polar_500_i_2013 = np.reshape(rho_500_i, (nt, nphi, 4))
    #
    # rho_polar = np.concatenate(
    #     (rho_polar_400_i_2013, rho_polar_400_i_2014), axis=2)

    # rho_polar_2013 = rho_polar_400_i_2013
    # rho_polar_2013 = np.concatenate(
    #     (rho_polar_400_i_2013, rho_polar_500_i_2013), axis=2)

    # rho_polar_2014 = rho_polar_400_i_2014

    # rho_list1.append(rho_polar_400_i_2013)
    # rho_list2.append(rho_polar_400_i_2014)
    rho_list1.append(rho_polar_500_i_2013)



# rho1 = np.array(rho_list1)
# rho2 = np.array(rho_list2)
# rho = np.concatenate((rho1, rho2), axis=0)  # Shape: (5840, 19, 24, 4)

rho = np.array(rho_list1)

rho_zeros = np.zeros((2920, 20, 24, 4))  # 2920, 20, 24, 4 
rho_zeros[:, :nt, :nphi, :] = rho
del rho_list1, rho  #, rho_list2, rho1, rho2

training_data = rho_zeros[:2000] # 2000
validation_data = rho_zeros[2000:]

training_data_resh = np.reshape(training_data, newshape=(2000, 20*24*4))
nPoints_val = len(rho_zeros) - len(training_data)
validation_data_resh = np.reshape(validation_data, newshape=(nPoints_val, 20*24*4))


In [20]:
def cosine_kernel(x, y):
    x_normalized = normalize(x, copy=True)
    if x is y:
        y_normalized = x_normalized
    else:
        y_normalized = normalize(y, copy=True)
    kernels = np.dot(x_normalized, y_normalized.T)
    return kernels


def my_kernel(X, Y):
    # The best one so far: np.dot(np.sin(X), np.sin(Y).T). It may be better than the linear one if a nonlinear
    # pre-image would exist
    return np.dot(np.sin(X), np.sin(Y).T)


In [3]:
nPoints_val = 920  # 840 920
validation_data_resh = np.reshape(validation_data, newshape=(nPoints_val, 20*24*4))
rhoavg = np.mean(validation_data_resh, axis=0)  # Compute mean
rho_msub_val = validation_data_resh.T - np.tile(rhoavg, (nPoints_val, 1)).T  # Mean-subtracted data

# print(training_data_resh.shape)
rhoavg = np.mean(training_data_resh, axis=0)  # Compute mean
nPoints = 2000 # 2000
rho_msub = training_data_resh.T - np.tile(rhoavg, (nPoints, 1)).T  # Mean-subtracted data
num_modes = 10

In [3]:
lorenz0_tr = pd.read_csv('output/training_data0.txt')
lorenz0_vl = pd.read_csv('output/validation_data0.txt')

lorenz0_np_tr = pd.DataFrame.to_numpy(lorenz0_tr)
lorenz0_np = pd.DataFrame.to_numpy(lorenz0_vl)

decoded0_df = pd.read_csv('output/decoded0.txt')
decoded0_np = pd.DataFrame.to_numpy(decoded0_df)


In [4]:
lorenz0_np_tr.shape

(255999, 128)

The most suitable Kernel: 

$K(X_1, X_2) = e^{(-\gamma ||X_1 - X_2||^2)}$

In [14]:
def mykpca(x):
    try: 
        kpca_lor = KernelPCA(n_components=3, kernel="rbf", fit_inverse_transform=True, gamma=x[0],
                     alpha=x[1])  #  1e-11
        X_kpca_lor = kpca_lor.fit_transform(lorenz0_np)
        X_back_kpca_lor = kpca_lor.inverse_transform(X_kpca_lor)
        error_adv = lorenz0_np-X_back_kpca_lor
        error_norm_adv = linalg.norm(error_adv)
        return error_norm_adv
    except:
        return 1e+10
gamma_init = np.linspace(0.01,0.6,num=5)
alpha_init = np.linspace(1e-9, 1e-5, num=5)
res_x = []
res_value = []
for i in range(5):
    x0 = [gamma_init[i], alpha_init[i]] # alpha_init[i]
    # bounds = [(0.2, 0.4), (1e-14, 1e-12)]
    res = minimize(mykpca, x0, method='Nelder-Mead', tol=1e-14, options={'maxiter':50, 'disp': True})
    # res = differential_evolution(mykpca, bounds, maxiter=10, tol=1e-14, disp=True)
    res_x.append(res.x)
    res_value.append(res.fun)
print(res_x)
print(res_value)

c:\My_folder\Documents\Politecnico di Milano\My thesis\test_auto\HASDM_ROM\venv\lib\site-packages\sklearn\decomposition\_kernel_pca.py:409: LinAlgWarning: Ill-conditioned matrix (rcond=8.28907e-17): result may not be accurate.
  self.dual_coef_ = linalg.solve(K, X, sym_pos=True, overwrite_a=True)
c:\My_folder\Documents\Politecnico di Milano\My thesis\test_auto\HASDM_ROM\venv\lib\site-packages\sklearn\decomposition\_kernel_pca.py:409: LinAlgWarning: Ill-conditioned matrix (rcond=8.28907e-17): result may not be accurate.
  self.dual_coef_ = linalg.solve(K, X, sym_pos=True, overwrite_a=True)
c:\My_folder\Documents\Politecnico di Milano\My thesis\test_auto\HASDM_ROM\venv\lib\site-packages\sklearn\decomposition\_kernel_pca.py:409: LinAlgWarning: Ill-conditioned matrix (rcond=5.71867e-17): result may not be accurate.
  self.dual_coef_ = linalg.solve(K, X, sym_pos=True, overwrite_a=True)
c:\My_folder\Documents\Politecnico di Milano\My thesis\test_auto\HASDM_ROM\venv\lib\site-packages\sklearn\

[array([6.88349557e-03, 1.81877613e-13]), array([3.50119169e-03, 3.71320461e-11]), array([6.87100506e-03, 9.61876726e-10]), array([5.98371471e-03, 1.03876643e-09]), array([5.55973291e-03, 1.28090382e-10])]
[0.11958095343710647, 0.3969503780092596, 0.4373236143938803, 0.5275187893659179, 0.2924873450896861]


In [17]:
print(res_x)

[array([1.13953125e-02, 1.04687500e-09])]


In [4]:
# lorenz0_np = lorenz0_np[:1000, :]
# lorenz0_np_tr = lorenz0_np_tr[:1000, :]
# decoded0_np = decoded0_np[:1000, :]

error_auto_lor = lorenz0_np - decoded0_np
error_norm_auto_lor = linalg.norm(error_auto_lor)
print('error_norm_autoencoder_lorenz:', error_norm_auto_lor)
# error_auto_lor = lorenz0_np_tr - decoded0_np
# error_norm_auto_lor = linalg.norm(error_auto_lor)
# print('error_norm_autoencoder_lorenz_tr:', error_norm_auto_lor)

pca = PCA(n_components=3)
X_pca_lin_lor = pca.fit_transform(lorenz0_np)
X_back_lin_lor = pca.inverse_transform(X_pca_lin_lor)
error_adv = lorenz0_np-X_back_lin_lor
error_norm_adv = linalg.norm(error_adv)
print('error_norm_lorenz:', error_norm_adv)
x = [6.88349557e-03, 1.81877613e-13]
kpca_lor = KernelPCA(n_components=3, kernel="rbf", fit_inverse_transform=True, gamma=x[0],
                     alpha=x[1])  #  0.011, 1e-11
X_kpca_lor = kpca_lor.fit_transform(lorenz0_np)
X_back_kpca_lor = kpca_lor.inverse_transform(X_kpca_lor)
error_adv = lorenz0_np-X_back_kpca_lor
error_norm_adv = linalg.norm(error_adv)
print('error_norm_lorenz_kpca:', error_norm_adv)


NameError: name 'lorenz0_np' is not defined

In [5]:
from scipy.optimize import minimize, differential_evolution
def mykpca(x):
    try: 
        kpca = KernelPCA(n_components=10, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])
        X_pca = kpca.fit_transform(rho_msub.T)
        X_back = kpca.inverse_transform(X_pca)
        X_back = X_back.T
        error_adv = rho_msub-X_back
        error_norm_adv = linalg.norm(error_adv)
        return error_norm_adv
    except:
        return 1e+10
gamma_init = np.linspace(0.1,0.6,num=5)
alpha_init = np.linspace(9.52148437e-14, 9.52148437e-9, num=5)
res_x = []
res_value = []
for i in range(5):
    x0 = [gamma_init[i], 4.07854175e-14] # alpha_init[i]
    # bounds = [(0.2, 0.4), (1e-14, 1e-12)]
    res = minimize(mykpca, x0, method='Nelder-Mead', tol=1e-14, options={'maxiter':50, 'disp': True})
    # res = differential_evolution(mykpca, bounds, maxiter=10, tol=1e-14, disp=True)
    res_x.append(res.x)
    res_value.append(res.fun)

asd


In [12]:
print(res_x)
print(res_value)

[array([3.22535513e-01, 5.49440980e-14]), array([3.26689453e-01, 4.66323895e-14]), array([3.27487842e-01, 4.21048114e-14]), array([3.25663757e-01, 4.49891733e-14]), array([3.25336048e-01, 4.91767092e-14])]
[0.12991970290535929, 0.1288634639190704, 0.1286555825700649, 0.12868331248343692, 0.12901360063077633]


In [30]:
# [array([3.28794335e-01, 4.07854175e-14]), array([1.82838673e-01, 9.81933441e-14])]
# [0.1284836616298345, 0.26848314372865445]

# x = [3.27369962e-01 4.12452062e-14] # res = minimize(mykpca, x0, method='Nelder-Mead', tol=1e-14, options={'maxiter':100, 'disp': True})

# x = [0.31, 7.45058089e-14]
# kpca = KernelPCA(n_components=10, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])
# X_pca = kpca.fit_transform(rho_msub.T)
# X_back = kpca.inverse_transform(X_pca)
# X_back = X_back.T
# error_adv = rho_msub-X_back
# error_norm_adv = linalg.norm(error_adv)
# print(error_norm_adv)

# x = [3.27369962e-01, 4.66323895e-14]
# x = np.linspace(1e-4, 10, num=10)
x = np.arange(-7,2, 0.25)
x = 10 ** x
# x = np.array([1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 2, 5, 10])
kpca_error = np.zeros((1, x.shape[0]))
for i in range(x.shape[0]):
    kpca1 = KernelPCA(n_components=10, kernel="rbf", fit_inverse_transform=True, gamma=x[i], alpha=4.66323895e-12)
    X_pca = kpca1.fit_transform(rho_msub.T)
    X_back = kpca1.inverse_transform(X_pca)
    X_back = X_back.T
    error_adv = rho_msub-X_back
    error_norm_adv = linalg.norm(error_adv)
    # print(error_norm_adv)
    kpca_error[:, i] = error_norm_adv
    del kpca1
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Kernel parameter $\gamma$")
plt.ylabel("$L^2$ norm of reconstruction error")
plt.semilogx(x, kpca_error.T, linewidth=2)
# plt.plot(kpca_error, linewidth=2) # , label="PCA error"
# plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/kpca_vs_gamma.png')

# error_adv = np.reshape(error_adv.T, newshape=(2000, 20, 24, 4))
# plt.figure()
# plt.rcParams.update({'font.size': 14})  # increase the font size
# mpl.rcParams['legend.fontsize'] = 15
# plt.xlabel("Longitude [deg]")
# plt.ylabel("Latitude [deg]")
# plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error_adv[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
#              cmap="inferno", levels=900)
# plt.colorbar()
# plt.grid(True)
# plt.tight_layout()
# plt.savefig('output/ReconstructionError_kpca_adv.png')

In [5]:
pca = PCA(n_components=num_modes)
pca.fit(rho_msub.T)
X_pca_lin = pca.fit_transform(rho_msub_val.T)
X_back_lin = pca.inverse_transform(X_pca_lin)
X_back_lin = X_back_lin.T
error_lin = rho_msub_val-X_back_lin
error_norm_lin = linalg.norm(error_lin)
print(error_norm_lin)
x = [0.2575, 4.66323895e-12] # 3.27369962e-01 4.66323895e-14
kpca1 = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])
kpca1.fit(rho_msub.T)
X_pca = kpca1.fit_transform(rho_msub_val.T)
X_back = kpca1.inverse_transform(X_pca)
X_back = X_back.T
error_adv = rho_msub_val-X_back
error_norm_adv = linalg.norm(error_adv)
print(error_norm_adv)

0.6185030848776232
0.41124215511636236


In [20]:
# kpca_adv = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=0.11,
#                      alpha=1e-11)  #  1e-11
x = [3.27369962e-01, 4.66323895e-12] # 4.66323895e-14
# num_modes = 1
modes = []
PCA_error = []
KPCA_error = []

for num_modes in range(2,15):
    modes.append(num_modes)
    kpca_adv = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])

    # fftout = fftpack.fftn(rho_msub.T)

    pca = PCA(n_components=num_modes)

    # pca_fft = PCA(n_components=num_modes)
    # pca.fit(rho_msub.T)
    # X_pca_lin = pca.fit_transform(rho_msub_val.T)
    X_pca_lin = pca.fit_transform(rho_msub.T)

    # X_pca_lin_fft = pca_fft.fit_transform(fftout.real)

    # gram = cosine_kernel(rho_msub.T, rho_msub.T)
    # X_pca_man = pcam.fit_transform(gram)
    # X_pca = kpca.fit_transform(rho_msub.T)
    # explained_variance_kpca = (kpca.eigenvalues_ ** 2) / (len(rho_msub.T) - 1)

    # print(np.sum(explained_variance_kpca))

    # kpca_test = KernelPCA(n_components=1920, kernel="rbf", fit_inverse_transform=True, gamma=0.11)  #  0.11

    # kpca_test.fit(rho_msub.T)
    # X_pca_test = kpca_test.fit_transform(rho_msub_val.T)
    # X_pca_test = kpca_test.fit_transform(rho_msub.T)
    # explained_variance_kpca_test = (kpca_test.eigenvalues_ ** 2) / (len(rho_msub.T) - 1)

    # print(np.sum(explained_variance_kpca)/np.sum(explained_variance_kpca_test))

    # kpca_adv.fit(rho_msub.T)
    # X_pca_adv = kpca_adv.fit_transform(rho_msub_val.T)
    X_pca_adv = kpca_adv.fit_transform(rho_msub.T)

    X_back_adv = kpca_adv.inverse_transform(X_pca_adv)

    X_back_lin = pca.inverse_transform(X_pca_lin)
    # X_back_lin_fft = pca_fft.inverse_transform(X_pca_lin_fft)
    # X_back_lin_fft_inv = fftpack.ifftn(X_back_lin_fft)

    # K = my_kernel(X_pca,X_pca)
    # K.flat[:: nPoints + 1] += 1
    # dual_coef_ = linalg.solve(K, rho_msub.T, sym_pos=True, overwrite_a=True)
    # K = my_kernel(X_pca, X_pca)
    # X_back_man_nonl = np.dot(K, dual_coef_).T

    X_back_lin = X_back_lin.T

    # X_back = X_back.T
    X_back_adv = X_back_adv.T

    # X_back_lin_fft_inv = X_back_lin_fft_inv.T

    # error = rho_msub-X_back
    # error_norm = linalg.norm(error)
    # print('error_norm_rbf:', error_norm)  # Error in reconstruction using built-in rbf kpca

    error_adv = rho_msub-X_back_adv
    error_norm_adv = linalg.norm(error_adv)
    # print('error_norm_rbf with input alpha:', error_norm_adv)  # Error in reconstruction using built-in rbf kpca
    KPCA_error.append(error_norm_adv) 


    error_lin = rho_msub-X_back_lin
    error_norm_lin = linalg.norm(error_lin)  # , ord=inf
    # print('error_norm_lin:', error_norm_lin)   # Error in reconstruction using built-in linear pca with
    PCA_error.append(error_norm_lin) 

    # error_lin_fft = fftout.real - X_back_lin_fft.T
    # error_lin_fft = rho_msub - X_back_lin_fft_inv
    # error_norm_lin_fft = linalg.norm(error_lin_fft)  # , ord=inf
    # print('error_norm_lin_fft:', error_norm_lin_fft)  # Error in reconstruction using built-in linear pca with


    # fftin_resh = fftout.real.reshape(2000, 20, 24, 4)
    # fftout_resh = error_lin_fft.reshape(2000, 20, 24, 4)
    # error = rho_msub - X_back_lin_fft_inv
    # error = np.reshape(error.T, newshape=(2000, 20, 24, 4))
    # Path("./output/").mkdir(parents=True, exist_ok=True)
    # plt.figure()
    # plt.rcParams.update({'font.size': 14})  # increase the font size
    # mpl.rcParams['legend.fontsize'] = 15
    # plt.xlabel("Longitude [deg]")
    # plt.ylabel("Latitude [deg]")
    # plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
    #              cmap="inferno", levels=900)
    # plt.colorbar()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig('output/fft_error.png')

    # error = rho_msub-X_back_lin
    # error = np.reshape(error.T, newshape=(2000, 20, 24, 4))
    # plt.figure()
    # plt.rcParams.update({'font.size': 14})  # increase the font size
    # mpl.rcParams['legend.fontsize'] = 15
    # plt.xlabel("Longitude [deg]")
    # plt.ylabel("Latitude [deg]")
    # plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
    #              cmap="inferno", levels=900)
    # plt.colorbar()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig('output/ReconstructionError_pca.png')

    # error_adv = rho_msub-X_back_adv
    # error_adv = np.reshape(error_adv.T, newshape=(2000, 20, 24, 4))
    # plt.figure()
    # plt.rcParams.update({'font.size': 14})  # increase the font size
    # mpl.rcParams['legend.fontsize'] = 15
    # plt.xlabel("Longitude [deg]")
    # plt.ylabel("Latitude [deg]")
    # plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error_adv[10, :19, :, 0])/training_data[10, :19, :, 0]*100,
    #              cmap="inferno", levels=900)
    # plt.colorbar()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig('output/ReconstructionError_kpca_adv.png')

    # plt.close('all')

data = {'number of modes':modes,'PCA error':PCA_error, 'KPCA error':KPCA_error}
df = pd.DataFrame(data)
print(df)

plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("# of modes")
plt.ylabel("Reconstruction error")
plt.plot(PCA_error, label="PCA error", linewidth=2)
plt.plot(KPCA_error, label="KPCA error", linewidth=2)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/PCA_vs_KPCA_error.png')
plt.close('all')




    number of modes  PCA error  KPCA error
0                 2  16.714446   27.560622
1                 3  11.601645   11.954918
2                 4   9.173239    8.782353
3                 5   7.596694    6.668534
4                 6   6.108573    4.950681
5                 7   4.723315    3.676183
6                 8   3.120358    2.441859
7                 9   1.551486    1.339403
8                10   1.229338    0.549276
9                11   1.019850    0.123205
10               12   0.848673    0.072738
11               13   0.717967    0.043660
12               14   0.612778    0.029549


In [9]:
for i in range(num_modes):
    plt.figure()
    plt.rcParams.update({'font.size': 14})  # increase the font size
    mpl.rcParams['legend.fontsize'] = 15
    plt.xlabel("Longitude [deg]")
    plt.ylabel("Latitude [deg]")
    # plt.contourf(np.rad2deg(phi), np.rad2deg(t), training_data[10, :19, :, 0] * max_rho, cmap="viridis", levels=900)
    # X_pca X_pca_adv
    plt.plot(X_pca_adv[:, i])
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('output/KPCA_mode%d.png'%i)
plt.close('all')

In [33]:
def build_model(hp):
        z_size = 10
        model = tf.keras.Sequential([
                layers.Flatten(),
                layers.Dense(20 * 24 * 4, activation='relu'),
                layers.Dense(units=hp.Int("units", min_value=450, max_value=650, step=20), activation='relu'),
                layers.Dense(units=hp.Int("units", min_value=50, max_value=250, step=20), activation='relu'),
                # layers.Dense(64, activation='tanh'),
                layers.Dense(32, activation='relu'),
                # layers.Dense(16, activation='relu'),
                layers.Dense(z_size, activation='relu'),           
                layers.Input(shape=(z_size)),
                layers.Dense(32, activation='relu'),
                layers.Dense(units=hp.Int("units", min_value=50, max_value=250, step=20), activation='relu'),
                layers.Dense(units=hp.Int("units", min_value=450, max_value=650, step=20), activation='relu'),
                layers.Dense(20 * 24 * 4, activation='relu')
        ])
        model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
        )
        return model

# build_model(kt.HyperParameters())
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective='val_loss',
    max_trials=20)
tuner.search(training_data_resh, training_data_resh, epochs=50, 
        validation_data=(validation_data_resh, validation_data_resh))
best_model = tuner.get_best_models()[0]

Trial 10 Complete [00h 02m 43s]
val_loss: 2120.924560546875

Best val_loss So Far: 2081.10009765625
Total elapsed time: 00h 25m 38s
INFO:tensorflow:Oracle triggered exit


In [34]:
best_model.build(input_shape=(None, 20 * 24 * 4))
best_model.summary()

# tuner.results_summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 1920)              3688320   
                                                                 
 dense_1 (Dense)             (None, 470)               902870    
                                                                 
 dense_2 (Dense)             (None, 470)               221370    
                                                                 
 dense_3 (Dense)             (None, 32)                15072     
                                                                 
 dense_4 (Dense)             (None, 10)                330       
                                                                 
 input_1 (InputLayer)        multiple                  0

In [10]:
class Autoencoder(Model):
    def __init__(self, z_size):
        super(Autoencoder, self).__init__()
        self.z_size = z_size
        self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                layers.Dense(20 * 24 * 4, activation='relu'),
                layers.Dense(512, activation='relu'),
                layers.Dense(128, activation='relu'),
                # layers.Dense(64, activation='tanh'),
                layers.Dense(32, activation='relu'),
                # layers.Dense(16, activation='relu'),
                layers.Dense(z_size, activation='relu')           
        ])
        self.decoder = tf.keras.Sequential([
                layers.Input(shape=(z_size)),
                layers.Dense(32, activation='relu'),
                layers.Dense(128, activation='relu'),
                layers.Dense(512, activation='relu'),
                layers.Dense(20 * 24 * 4, activation='relu')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


# print(X_pca.shape)      
# print(X_pca_val.shape)
# print(training_data_resh.shape)
# print(validation_data_resh.shape)
# exit()
num_modes = 10
run = True
if run:
    autoencoder = Autoencoder(num_modes)
    autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

    history = autoencoder.fit(training_data_resh, training_data_resh,
                    batch_size=5,  # play with me
                    epochs= 50,  # 20
                    shuffle=True,
                    validation_data=(validation_data_resh, validation_data_resh))

    loss = list(history.history.values())
    autoencoder.encoder.save('encoder')
    autoencoder.decoder.save('decoder')
else:
    encoder = tf.keras.models.load_model('encoder')
    decoder = tf.keras.models.load_model('decoder')


encoded = autoencoder.encoder(validation_data_resh).numpy()
decoded = autoencoder.decoder(encoded).numpy()
np.savetxt('output/atm/encoded.txt', encoded, delimiter=',')
np.savetxt('output/atm/training_data.txt', training_data_resh, delimiter=',')
np.savetxt('output/atm/validation_data.txt', validation_data_resh, delimiter=',')
np.savetxt('output/atm/decoded.txt', decoded, delimiter=',')

plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Number of Epoch")
plt.ylabel("Loss")
plt.plot(loss[0], label="Train", linewidth=2)
plt.plot(loss[1], label="Validation", linewidth=2)
plt.yscale("log")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/loss_atm.png')

error = -validation_data_resh + decoded
error = np.reshape(error, newshape=(nPoints_val, 20, 24, 4))
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error[10, :19, :, 0])/validation_data[10, :19, :, 0]*100,
                cmap="inferno", levels=900)
plt.colorbar()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/ReconstructionError_nn.png')
error_norm_nn = linalg.norm(decoded - validation_data_resh)  # , ord=inf
print('error_norm_nn:', error_norm_nn)

NameError: name 'training_data_resh' is not defined

In [11]:
errorae = [66.8464, 28.1242, 25.1675, 26.8981, 21.2226, 18.3057]
modes = [2, 4, 6, 8, 10, 12]
PCA_error = [16.714446, 9.173239, 6.108573, 3.120358, 1.229338, 0.848673]
KPCA_error = [25.821876, 8.507736, 4.921118, 2.429778, 0.286163, 0.072301]
datam = {'number of modes':modes,'PCA error':PCA_error, 'KPCA error':KPCA_error, 'Autoencoder erros':errorae}
df = pd.DataFrame(datam)
print(df)
plt.figure()
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Number of modes")
plt.ylabel("Reconstruction error")
plt.plot(modes, PCA_error, label="PCA", linewidth=2)
plt.plot(modes, KPCA_error, label="KPCA", linewidth=2)
plt.plot(modes, errorae, label="Autoencoder", linewidth=2)
plt.legend()
plt.grid(True)
plt.yscale('log')

plt.tight_layout()
plt.savefig('output/error_comparison.png')
plt.close('all')

   number of modes  PCA error  KPCA error  Autoencoder erros
0                2  16.714446   25.821876            66.8464
1                4   9.173239    8.507736            28.1242
2                6   6.108573    4.921118            25.1675
3                8   3.120358    2.429778            26.8981
4               10   1.229338    0.286163            21.2226
5               12   0.848673    0.072301            18.3057


In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
# import tensorflow as tf
# from tensorflow.keras import layers, losses
# from tensorflow.keras.models import Model
# import keras_tuner as kt
# import modred as mr
from sklearn.decomposition import PCA, KernelPCA
from scipy import linalg
from sklearn.preprocessing import normalize
# from scipy import fftpack
from pathlib import Path
from scipy.optimize import minimize, differential_evolution

mpl.use('Agg')

# density_df_400_2013 = pd.read_csv('Data/2013_HASDM_400-475KM.den', delim_whitespace=True,
#                                   header=None)
# density_np_400_2013 = pd.DataFrame.to_numpy(density_df_400_2013)
# del density_df_400_2013
# If you want to add more altitudes (increase the initial dimension of the problem), uncomment the followings:
density_df_500_2013 = pd.read_csv('Data/2013_HASDM_500-575KM.den', delim_whitespace=True,
                                  header=None)
density_np_500_2013 = pd.DataFrame.to_numpy(density_df_500_2013)
del density_df_500_2013

# If you want to add more data (increase the time instants), uncomment the followings:
# density_df_400_2014 = pd.read_csv('Data/2014_HASDM_400-475KM.den', delim_whitespace=True,
#                                   header=None)
# density_np_400_2014 = pd.DataFrame.to_numpy(density_df_400_2014)
# del density_df_400_2014

nt = 19
nphi = 24

t = np.linspace(-np.pi / 2, np.pi / 2, nt)
phi = np.linspace(0, np.deg2rad(345), nphi)

# max_rho1 = np.max(density_np_400_2013[:, 10])
# max_rho2 = np.max(density_np_400_2014[:, 10])
# max_rho = np.max(np.array([max_rho1, max_rho2]))
# density_np_400_2013[:, 10] = density_np_400_2013[:, 10] / max_rho
# density_np_400_2014[:, 10] = density_np_400_2014[:, 10] / max_rho

# max_rho = np.max(density_np_400_2013[:, 10])
max_rho = np.max(density_np_500_2013[:, 10])
# max_rho = np.max(np.array([max_rho2, max_rho]))

# density_np_400_2013[:, 10] = density_np_400_2013[:, 10] / max_rho
density_np_500_2013[:, 10] = density_np_500_2013[:, 10] / max_rho
# density_np_400_2014[:, 10] = density_np_400_2014[:, 10] / max_rho

rho_list = []
rho_list1 = []
rho_list2 = []

for i in range(int(1331520 / (nt * nphi))):  # 1335168
    # rho_400_i_2013 = density_np_400_2013[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    # rho_polar_400_i_2013 = np.reshape(rho_400_i_2013, (nt, nphi, 4))

    # rho_400_i_2014 = density_np_400_2014[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    # rho_polar_400_i_2014 = np.reshape(rho_400_i_2014, (nt, nphi, 4))

    rho_500_i = density_np_500_2013[i * (4 * nt * nphi):(i + 1) * (4 * nt * nphi), 10]
    rho_polar_500_i_2013 = np.reshape(rho_500_i, (nt, nphi, 4))
    #
    # rho_polar = np.concatenate(
    #     (rho_polar_400_i_2013, rho_polar_400_i_2014), axis=2)

    # rho_polar_2013 = rho_polar_400_i_2013
    # rho_polar_2013 = np.concatenate(
    #     (rho_polar_400_i_2013, rho_polar_500_i_2013), axis=2)

    # rho_polar_2014 = rho_polar_400_i_2014

    # rho_list1.append(rho_polar_400_i_2013)
    # rho_list2.append(rho_polar_400_i_2014)
    rho_list1.append(rho_polar_500_i_2013)



# rho1 = np.array(rho_list1)
# rho2 = np.array(rho_list2)
# rho = np.concatenate((rho1, rho2), axis=0)  # Shape: (5840, 19, 24, 4)

rho = np.array(rho_list1)

rho_zeros = np.zeros((2920, 20, 24, 4))  # 2920, 20, 24, 4 
rho_zeros[:, :nt, :nphi, :] = rho
del rho_list1, rho  #, rho_list2, rho1, rho2

training_data = rho_zeros[:2000] # 2000
validation_data = rho_zeros[2000:]

training_data_resh = np.reshape(training_data, newshape=(2000, 20*24*4))
nPoints_val = len(rho_zeros) - len(training_data)
validation_data_resh = np.reshape(validation_data, newshape=(nPoints_val, 20*24*4))


nPoints_val = 920  # 840 920
validation_data_resh = np.reshape(validation_data, newshape=(nPoints_val, 20*24*4))
rhoavg_val = np.mean(validation_data_resh, axis=0)  # Compute mean
rho_msub_val = validation_data_resh.T - np.tile(rhoavg_val, (nPoints_val, 1)).T  # Mean-subtracted data

# print(training_data_resh.shape)
rhoavg = np.mean(training_data_resh, axis=0)  # Compute mean
nPoints = 2000 # 2000
rho_msub = training_data_resh.T - np.tile(rhoavg, (nPoints, 1)).T  # Mean-subtracted data
num_modes = 10

# def mykpca(x):
#     try: 
#         kpca = KernelPCA(n_components=10, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=4.66323895e-12)
#         X_pca = kpca.fit_transform(rho_msub.T)
#         X_back = kpca.inverse_transform(X_pca)
#         X_back = X_back.T
#         error_adv = rho_msub-X_back
#         error_norm_adv = linalg.norm(error_adv)
#         return error_norm_adv
#     except:
#         return 1e+10
# gamma_init = np.linspace(0.1,0.6,num=5)
# alpha_init = np.linspace(9.52148437e-14, 9.52148437e-9, num=5)
# res_x = []
# res_value = []
# for i in range(1):
#     x0 = gamma_init[i] # alpha_init[i]
#     # bounds = [(0.2, 0.4), (1e-14, 1e-12)]
#     res = minimize(mykpca, x0, method='Nelder-Mead', tol=1e-13, options={'maxiter':10, 'disp': True})
#     # res = differential_evolution(mykpca, bounds, maxiter=10, tol=1e-14, disp=True)
#     res_x.append(res.x)
#     res_value.append(res.fun)
# print(res_x)



In [2]:
x = [0.2575, 4.66323895e-12] # 3.27369962e-01 4.66323895e-14
kpca1 = KernelPCA(n_components=num_modes, kernel="rbf", fit_inverse_transform=True, gamma=x[0], alpha=x[1])
kpca1.fit(rho_msub.T)
X_pca = kpca1.fit_transform(rho_msub_val.T)
X_back = kpca1.inverse_transform(X_pca)
X_back = X_back.T
error_adv = rho_msub_val-X_back
error_norm_adv = linalg.norm(error_adv)
print(error_norm_adv)
error_adv = np.reshape(error_adv, newshape=(nPoints_val, 20, 24, 4))
X_back = X_back + np.tile(rhoavg_val, (nPoints_val, 1)).T
X_back_kpca = np.reshape(X_back, newshape=(nPoints_val, 20, 24, 4))

print(np.absolute(error_adv[0, :19, :, 0]).max())
print(validation_data[0, :19, :, 0].max())
err = np.absolute(error_adv[0, :19, :, 0])/validation_data[0, :19, :, 0]
print(err.max())


0.41124215511636236
0.0013024720672199785
0.22423398328690808
0.04276640939152378


In [17]:
plt.close("all")
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error_adv[0, :19, :, 0])/validation_data[0, :19, :, 0]*100,
                cmap="inferno", levels=900)
plt.colorbar(label="[%]")
plt.grid(True)
plt.tight_layout()
plt.savefig('output/ReconstructionError_kpca.png')
print(np.absolute(error_adv).max())

plt.figure() 
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), validation_data[0, :19, :, 0]*max_rho,
                cmap="inferno", levels=900)
plt.colorbar()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/dens_input.png')
plt.figure() 
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), X_back_kpca[0, :19, :, 0]*max_rho,
                cmap="inferno", levels=900)
plt.colorbar()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/recons_kpca.png')

pca = PCA(n_components=num_modes)
pca.fit(rho_msub.T)
X_pca_lin = pca.fit_transform(rho_msub_val.T)
X_back_lin = pca.inverse_transform(X_pca_lin)
X_back_lin = X_back_lin.T
error_lin = rho_msub_val-X_back_lin
error_lin = np.reshape(error_lin, newshape=(nPoints_val, 20, 24, 4))

error_norm_lin = linalg.norm(error_lin)
plt.figure() 
plt.rcParams.update({'font.size': 14})  # increase the font size
mpl.rcParams['legend.fontsize'] = 15
plt.xlabel("Longitude [deg]")
plt.ylabel("Latitude [deg]")
plt.contourf(np.rad2deg(phi), np.rad2deg(t), np.absolute(error_lin[0, :19, :, 0])/validation_data[0, :19, :, 0]*100,
                cmap="inferno", levels=900)
plt.colorbar(label="[%]")
plt.grid(True)
plt.tight_layout()
plt.savefig('output/ReconstructionError_pca.png')
print(np.absolute(error_lin).max())


0.008016703025546718
0.007385226171594428
